<a href="https://colab.research.google.com/github/adammoss/bnn_hmc/blob/main/results/CMD_MCD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import os
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
samples_iter = 200
dropout = 0.1

In [3]:
!pip install astro-datasets --upgrade
!pip install tensorflow_datasets --upgrade
import astro_datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
ds_train, info_train = tfds.load(name='cmd', split='train[:90%]', with_info=True, batch_size=75, as_supervised=True, builder_kwargs={'simulation': 'IllustrisTNG', 'field': 'Mtot', 'parameters': ['omegam']})
ds_valid, info_valid = tfds.load(name='cmd', split='train[90%:95%]', with_info=True, batch_size=75, as_supervised=True, builder_kwargs={'simulation': 'IllustrisTNG', 'field': 'Mtot','parameters': ['omegam']})
ds_test, info_test = tfds.load(name='cmd', split='train[95%:]', with_info=True, batch_size=75, as_supervised=True, builder_kwargs={'simulation': 'IllustrisTNG', 'field': 'Mtot','parameters': ['omegam']})

In [5]:
minimum = np.array([0.1, 0.6])
maximum = np.array([0.5, 1.0])  

def normalize(image, label):  
  image = tf.math.log(image)
  label = (label - minimum[0])/(maximum[0] - minimum[0])
  return image, label

ds_train = ds_train.map(normalize)
ds_valid = ds_valid.map(normalize)
ds_test = ds_test.map(normalize)

def rotate_map(image, label):
    return tf.image.rot90(image, np.random.choice([-1,0,1,2])), label

#ds_train = ds_train.map(rotate_map)

In [6]:
class MonteCarloDropout(tf.keras.layers.Dropout):
     def call(self, inputs):
         return super().call(inputs, training=True)

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(6, kernel_size=5, strides=1, padding = 'same', input_shape=(256, 256, 1)),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2,2)),
    MonteCarloDropout(dropout),
    tf.keras.layers.Conv2D(16, kernel_size=5, strides=1, padding = 'same'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2,2)),
    MonteCarloDropout(dropout),
    tf.keras.layers.Conv2D(120, kernel_size=5, strides=1, padding = 'same'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPool2D(pool_size=(3, 3), strides=(2,2)),
    MonteCarloDropout(dropout),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(84),
    tf.keras.layers.ReLU(),
    MonteCarloDropout(dropout),
    tf.keras.layers.Dense(1, activation='linear'),
    ])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 6)       156       
                                                                 
 re_lu (ReLU)                (None, 256, 256, 6)       0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 6)      0         
 )                                                               
                                                                 
 monte_carlo_dropout (MonteC  (None, 127, 127, 6)      0         
 arloDropout)                                                    
                                                                 
 conv2d_1 (Conv2D)           (None, 127, 127, 16)      2416      
                                                                 
 re_lu_1 (ReLU)              (None, 127, 127, 16)      0

In [9]:
opt = tf.keras.optimizers.Adam(learning_rate=5e-5)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.95, patience=3)

In [10]:
model.compile(optimizer=opt,
              loss='MeanSquaredError',
              metrics='mae')

In [11]:
checkpoint_path = "cmd/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                  save_weights_only=False,
                                                  monitor='val_loss',
                                                  mode='min',
                                                  verbose=1,
                                                  save_best_only=True)

In [12]:

model.fit(ds_train, epochs=100, callbacks=[reduce_lr, cp_callback], validation_data=ds_valid)

Epoch 1/100
179/180 [============================>.] - ETA: 0s - loss: 6.3888 - mae: 0.7144
Epoch 1: val_loss improved from inf to 0.10846, saving model to cmd/cp.ckpt
180/180 [==============================] - 19s 33ms/step - loss: 6.3538 - mae: 0.7119 - val_loss: 0.1085 - val_mae: 0.2787 - lr: 5.0000e-05
Epoch 2/100
178/180 [============================>.] - ETA: 0s - loss: 0.1010 - mae: 0.2665
Epoch 2: val_loss improved from 0.10846 to 0.10205, saving model to cmd/cp.ckpt
180/180 [==============================] - 6s 32ms/step - loss: 0.1010 - mae: 0.2665 - val_loss: 0.1021 - val_mae: 0.2715 - lr: 5.0000e-05
Epoch 3/100
178/180 [============================>.] - ETA: 0s - loss: 0.0994 - mae: 0.2636
Epoch 3: val_loss did not improve from 0.10205
180/180 [==============================] - 4s 24ms/step - loss: 0.0994 - mae: 0.2635 - val_loss: 0.1044 - val_mae: 0.2710 - lr: 5.0000e-05
Epoch 4/100
180/180 [==============================] - ETA: 0s - loss: 0.1006 - mae: 0.2653
Epoch 4: va

In [18]:
new_model = tf.keras.models.load_model('cmd/cp.ckpt')

In [19]:
def unnormalize(image, label):  
  image = tf.math.exp(image)
  label = (label*(maximum[0] - minimum[0])) + minimum[0]
  return image, label

In [20]:
for _ in range(10):
  acc = 0
  for (x, y) in ds_test:
    yhat = new_model.predict(x)
    yhat = (yhat*(maximum[0] - minimum[0])) + minimum[0]
    y = (y*(maximum[0] - minimum[0])) + minimum[0]
    acc += np.mean(np.abs((yhat-y)/y))
    
  print('Inaccuracy: ', acc)

Inaccuracy:  1.9566980130478107
Inaccuracy:  1.9506650962432812
Inaccuracy:  1.9758019437204397
Inaccuracy:  1.952405714825556
Inaccuracy:  1.9487364918593832
Inaccuracy:  1.962029045440706
Inaccuracy:  1.970165252422194
Inaccuracy:  2.0003404120897224
Inaccuracy:  1.9866567942775726
Inaccuracy:  1.9348364792438373


In [21]:
ds_test_S, info_test_S = tfds.load(name='cmd', split='train[95%:]', with_info=True, batch_size=75, as_supervised=True, builder_kwargs={'simulation': 'SIMBA', 'field': 'Mtot','parameters': ['omegam']})
ds_test_S = ds_test_S.map(normalize)

In [22]:
for _ in range(10):
  acc = 0
  for (x, y) in ds_test_S:
    yhat = new_model.predict(x)
    yhat = (yhat*(maximum[0] - minimum[0])) + minimum[0]
    y = (y*(maximum[0] - minimum[0])) + minimum[0]
    acc += np.mean(np.abs((yhat-y)/y))
    
  print('Inaccuracy: ', acc)

Inaccuracy:  2.112305564429031
Inaccuracy:  2.1328012406331243
Inaccuracy:  2.126162596757259
Inaccuracy:  2.0840792959890226
Inaccuracy:  2.0313658665162615
Inaccuracy:  2.1843905538808674
Inaccuracy:  2.146604992527698
Inaccuracy:  2.1763182556852945
Inaccuracy:  2.1941260936263016
Inaccuracy:  2.147018096779938
